In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.environ['VLLM_USE_V1'] = '0'
os.environ["NVTE_ALLOW_NONDETERMINISTIC_ALGO"] = "1"

import torch
import vllm
from vllm import LLM, SamplingParams
from vllm.model_executor import SamplingMetadata
from vllm.model_executor.layers.logits_processor import _prune_hidden_states
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm


import inference_verification.data_utils as data_utils

/workspace/inference_verification/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-11 02:36:12 [__init__.py:239] Automatically detected platform cuda.


2025-06-11 02:36:13,839	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
model_name = "google/gemma-2-2b-it"
dtype = "bfloat16"
dataset_name = "lmsys/lmsys-chat-1m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
ctx_len = 1024

max_decode_tokens = 1024
max_decode_tokens = 32
n_samples = 10

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, ignore_eos=True, max_tokens=max_decode_tokens + 1)
sampling_params = SamplingParams(temperature=0.0, ignore_eos=True, max_tokens=max_decode_tokens+1)

In [4]:

ds = load_dataset(dataset_name, split="train")
prompts = [i['conversation'] for _, i in zip(range(n_samples), ds)]

prompts = [tokenizer.apply_chat_template(prompt, tokenize=False) for prompt in prompts]

tokenized_inputs = tokenizer(prompts, padding=False, return_tensors=None, add_special_tokens=False, truncation=True, max_length=ctx_len)
prompt_token_ids = [input_ids for input_ids in tokenized_inputs["input_ids"]]

for i in range(len(prompt_token_ids)):
    print(len(prompt_token_ids[i]))



print(prompts[0])

259
1024
359
165
210
220
320
388
701
704
<bos><start_of_turn>user
how can identity protection services help protect me against identity theft<end_of_turn>
<start_of_turn>model
Identity protection services can help protect you against identity theft in several ways:

1. Monitoring: Many identity protection services monitor your credit reports, public records, and other sources for signs of identity theft. If they detect any suspicious activity, they will alert you so you can take action.
2. Credit freeze: Some identity protection services can help you freeze your credit, which makes it more difficult for thieves to open new accounts in your name.
3. Identity theft insurance: Some identity protection services offer insurance that can help you recover financially if you become a victim of identity theft.
4. Assistance: Many identity protection services offer assistance if you become a victim of identity theft. They can help you file a police report, contact credit bureaus, and other steps

In [5]:
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    max_model_len=ctx_len*2,
    enforce_eager=True,
    dtype=dtype,
    disable_async_output_proc=True,
)
model = llm.llm_engine.model_executor.driver_worker.model_runner.model

INFO 06-11 02:36:22 [config.py:717] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
INFO 06-11 02:36:22 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='google/gemma-2-2b-it', speculative_config=None, tokenizer='google/gemma-2-2b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.07it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  2.06it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.80it/s]


INFO 06-11 02:36:25 [loader.py:458] Loading weights took 1.12 seconds


INFO 06-11 02:36:25 [model_runner.py:1140] Model loading took 4.9000 GiB and 1.404967 seconds
INFO 06-11 02:36:27 [worker.py:287] Memory profiling takes 1.18 seconds
INFO 06-11 02:36:27 [worker.py:287] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.90) = 21.23GiB
INFO 06-11 02:36:27 [worker.py:287] model weights take 4.90GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.34GiB; the rest of the memory reserved for KV Cache is 13.93GiB.
INFO 06-11 02:36:27 [executor_base.py:112] # cuda blocks: 8779, # CPU blocks: 2520
INFO 06-11 02:36:27 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 68.59x
INFO 06-11 02:36:31 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 5.17 seconds


In [6]:

temp_saved_activations = []
def activation_saving_hook(module, input, output):
    temp_saved_activations.append(output[0].detach().clone())



# def toploc_cache_hook(_, inputs: tuple, topk_cache: list):
#     """
#     Pre-hook to get final hidden states of a model forward pass and add it to
#     the TOPLOC cache based on sampling metadata.


#     Args:
#         _ (): Unused argument, required for compatibility with hook function signature.
#         input (tuple): A tuple containing the input data. The first element is expected to be the hidden states of the model, and the second element is expected to be the sampling metadata.
#         toploc_cache (TopLocCache): The TopLocCache instance to which the processed data will be added.
#     """
#     print(len(inputs))
#     print(inputs[0].shape)
#     print(inputs[1].shape)
#     # Get hidden states and sampling metadata from inputs
#     hidden_states, sampling_metadata = inputs[1], inputs[2]
#     assert isinstance(hidden_states, torch.Tensor)
#     assert isinstance(sampling_metadata, SamplingMetadata)

#     # This check is true only for prefills
#     if max(sampling_metadata.selected_token_indices) > len(sampling_metadata.seq_groups):
#         return

#     # This pruning is required when cuda graph padding is enabled.
#     hidden_states = _prune_hidden_states(hidden_states, sampling_metadata)
#     if len(sampling_metadata.seq_groups) != hidden_states.shape[0]:
#         raise ValueError(f"Lengths dont match: {len(sampling_metadata.seq_groups)} {hidden_states.shape}")

#     # Get seq_ids from seq_groups
#     seq_ids = [seq_group.seq_ids[0] for seq_group in sampling_metadata.seq_groups]

#     # Add hidden states to TOPLOC cache with corresponding seq_ids
#     topk_cache.append((hidden_states, seq_ids))

In [7]:
saved_activations_handle = model.model.norm.register_forward_hook(activation_saving_hook)
# saved_activations_handle = model.model.norm.register_forward_hook(toploc_cache_hook)

temp_saved_activations = []
orig_activations = []

prefill_tokens = []


sampling_params = SamplingParams(temperature=0.0, ignore_eos=True, max_tokens=max_decode_tokens+1)

try:
    for i in tqdm(range(len(prompt_token_ids)), desc="Generating activations"):
        outputs = llm.generate(prompt_token_ids=prompt_token_ids[i], sampling_params=sampling_params, use_tqdm=False)
        orig_activations.append(temp_saved_activations)
        temp_saved_activations = []
        # print(outputs)
        # print(torch.tensor(outputs[0].outputs[0].token_ids).shape)
        prefill_tokens.append(prompt_token_ids[i] + list(outputs[0].outputs[0].token_ids))
except Exception as e:
    print(e)
finally:
    saved_activations_handle.remove()


Generating activations:   0%|          | 0/10 [00:00<?, ?it/s]

Generating activations: 100%|██████████| 10/10 [00:05<00:00,  1.69it/s]


In [8]:
def concatenate_activations(orig_activations: list[list[torch.Tensor]]) -> list[torch.Tensor]:
    concatenated = []
    for sample_activations in orig_activations:
        # Concatenate all activations along the sequence dimension (dim=0)
        concat_tensor = torch.cat(sample_activations, dim=0)
        concatenated.append(concat_tensor)
    
    return concatenated

orig_activations = concatenate_activations(orig_activations)

In [9]:
for i in range(len(prefill_tokens)):
    print(len(prefill_tokens[i]), len(orig_activations[i]), len(prompt_token_ids[i]))
    print(len(prefill_tokens[i]) - len(prompt_token_ids[i]))

292 291 259
33
1057 1056 1024
33
392 391 359
33
198 197 165
33
243 242 210
33
253 252 220
33
353 352 320
33
421 420 388
33
734 733 701
33
737 736 704
33


In [10]:
saved_activations_handle = model.model.norm.register_forward_hook(activation_saving_hook)

temp_saved_activations = []
prefill_activations = []


sampling_params = SamplingParams(temperature=0.0, ignore_eos=True, max_tokens=1)

try:
    for i in tqdm(range(len(prefill_tokens)), desc="Generating activations"):
        outputs = llm.generate(prompt_token_ids=prefill_tokens[i], sampling_params=sampling_params, use_tqdm=False)
        # remove the list, -1 because that wasn't in orig_activations
        prefill_activations.append(temp_saved_activations[0][:-1])
        temp_saved_activations = []
except Exception as e:
    print(e)
finally:
    saved_activations_handle.remove()


Generating activations:   0%|          | 0/10 [00:00<?, ?it/s]

Generating activations: 100%|██████████| 10/10 [00:00<00:00, 25.68it/s]


In [11]:
# print(orig_activations[0][0].shape)
# print(orig_activations[0][1].shape)
# print(orig_activations[0][2].shape)
# print(len(orig_activations[0]))
# print(len(orig_activations))

# for i in range(len(orig_activations[0])):
#     print(orig_activations[0][i].shape, i)


In [12]:
print(prefill_activations[0].shape)
print(orig_activations[0].shape)
print(len(prefill_activations))
print(len(orig_activations))

torch.Size([291, 2304])
torch.Size([291, 2304])
10
10


In [13]:
import torch

def compress_activations(activations, k=100):
    """
    Compress activations by keeping only the top-k absolute values and their indices.
    
    Args:
        activations: torch tensor of shape (num_samples, hidden_dim)
        k: number of top values to keep per sample
    
    Returns:
        dict with 'values' and 'indices' tensors, both of shape (num_samples, k)
    """
    # Get top-k values by absolute magnitude
    abs_activations = torch.abs(activations)
    top_k_values, top_k_indices = torch.topk(abs_activations, k=k, dim=1)
    
    # Get the actual values (with sign preserved) using gather
    actual_values = torch.gather(activations, 1, top_k_indices).to(device=activations.device)
    
    return {
        'values': actual_values,
        'indices': top_k_indices
    }


def compute_euclidean_similarity(compressed1, compressed2):
    """
    Compute Euclidean-based similarity between two compressed activation representations.
    
    Args:
        compressed1, compressed2: dicts with 'values' and 'indices' tensors
    
    Returns:
        similarity scores for each sample (higher is more similar)
    """
    batch_size = compressed1['values'].shape[0]
    hidden_dim = compressed1['indices'].max().item() + 1  # Assuming indices are 0-based
    
    similarities = torch.zeros(batch_size)
    
    for i in range(batch_size):
        # Create sparse representations
        sparse1 = torch.zeros(hidden_dim)
        sparse2 = torch.zeros(hidden_dim)
        
        # Fill in the values at their respective indices
        sparse1.scatter_(0, compressed1['indices'][i], compressed1['values'][i])
        sparse2.scatter_(0, compressed2['indices'][i], compressed2['values'][i])
        
        # Compute Euclidean distance and convert to similarity
        distance = torch.norm(sparse1 - sparse2, p=2)
        similarities[i] = 1 / (1 + distance)  # Convert distance to similarity
    
    return similarities


# Alternative: Batch-efficient version
def compute_euclidean_similarity_batch(compressed1, compressed2, hidden_dim=None):
    """
    Batch-efficient version of Euclidean similarity computation.
    
    Args:
        compressed1, compressed2: dicts with 'values' and 'indices' tensors
        hidden_dim: dimension of the hidden layer (if None, inferred from indices)
    
    Returns:
        similarity scores for each sample
    """
    batch_size, k = compressed1['values'].shape
    
    if hidden_dim is None:
        hidden_dim = max(compressed1['indices'].max().item(), 
                         compressed2['indices'].max().item()) + 1
    
    # Create sparse representations for the entire batch
    sparse1 = torch.zeros(batch_size, hidden_dim)
    sparse2 = torch.zeros(batch_size, hidden_dim)
    
    # Use scatter to fill in values
    batch_indices = torch.arange(batch_size).unsqueeze(1).expand(-1, k)
    sparse1.scatter_(1, compressed1['indices'], compressed1['values'])
    sparse2.scatter_(1, compressed2['indices'], compressed2['values'])
    
    # Compute Euclidean distances for all samples at once
    distances = torch.norm(sparse1 - sparse2, p=2, dim=1)
    similarities = 1 / (1 + distances)
    
    return similarities


# Example usage:
# Assuming your activations are already torch tensors
# If they're numpy arrays, convert them first:
# prefill_activations = torch.from_numpy(prefill_activations)
# orig_activations = torch.from_numpy(orig_activations)

# Compress your activations
prefill_compressed = compress_activations(torch.cat(prefill_activations, dim=0), k=100)
orig_compressed = compress_activations(torch.cat(orig_activations, dim=0), k=100)

# Compute similarities (use batch version for efficiency)
similarities = compute_euclidean_similarity_batch(prefill_compressed, orig_compressed)

print(f"Average similarity: {similarities.mean():.4f}")
print(f"Min similarity: {similarities.min():.4f}")
print(f"Max similarity: {similarities.max():.4f}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA_scatter__src)